In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import models,layers,optimizers,callbacks
from tensorflow.keras.models import load_model, save_model

2023-11-07 14:56:40.514084: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-07 14:56:40.877966: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-07 14:56:41.784470: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/aaron/anaconda3/envs/tf/lib/
2023-11-07 14:56:41.784599: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.

In [2]:
# Read data from a CSV file into a Pandas DataFrame
df = pd.read_csv("hearing_test.csv")

# Display the first few rows of the DataFrame
df.head()

,age,physical_score,test_result
0,33.0,40.7,1
1,50.0,37.2,1
2,52.0,24.7,0
3,56.0,31.0,0
4,35.0,42.9,1


In [3]:
# Check if there are any columns with all null values in the DataFrame
df.isnull().all()

age               False
physical_score    False
test_result       False
dtype: bool

In [4]:
# Extracting features and target from the DataFrame
features = df[["age", "physical_score"]]
target = df["test_result"]

In [5]:
# Initialize MinMaxScaler for scaling features to a specified range
scaler = MinMaxScaler()

# Scaling the selected features using the MinMaxScaler
features_scaled = scaler.fit_transform(features)

In [6]:
# Splitting the scaled features and target into training and validation sets
features_train, features_val, target_train, target_val = train_test_split(features_scaled, target, test_size=0.2, random_state=42)

In [7]:
# Create a sequential model
model = models.Sequential()

# Add LSTM layers to the model with specified parameters
model.add(layers.LSTM(64, input_shape=(features.shape[1], 1), activation='relu', return_sequences=True))
model.add(layers.Dropout(0.1))
model.add(layers.LSTM(32, activation='relu', return_sequences=True))
model.add(layers.Dropout(0.1))
model.add(layers.LSTM(16, activation='relu'))
model.add(layers.Dropout(0.1))

# Add dense layers to the model
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model with specified optimizer, loss function, and metrics
model.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping to prevent overfitting
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model on the training data, with validation data and early stopping
history = model.fit(features_train, target_train, epochs=50, batch_size=32, validation_data=(features_val, target_val), callbacks=[early_stopping])

# Evaluate the model on the validation set and print the accuracy
accuracy = model.evaluate(features_val, target_val)[1]
print("Validation Accuracy:", accuracy)

2023-11-07 14:58:57.458522: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-07 14:58:57.494277: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-07 14:58:57.494565: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-07 14:58:57.495283: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Epoch 1/50
 17/125 [===>..........................] - ETA: 0s - loss: 0.6847 - accuracy: 0.6507

2023-11-07 14:59:01.608834: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


125/125 [==============================] - 5s 15ms/step - loss: 0.6792 - accuracy: 0.5960 - val_loss: 0.6662 - val_accuracy: 0.6160
Epoch 2/50
125/125 [==============================] - 1s 12ms/step - loss: 0.6724 - accuracy: 0.5960 - val_loss: 0.6487 - val_accuracy: 0.6160
Epoch 3/50
125/125 [==============================] - 2s 12ms/step - loss: 0.4988 - accuracy: 0.7500 - val_loss: 0.3635 - val_accuracy: 0.8550
Epoch 4/50
125/125 [==============================] - 2s 13ms/step - loss: 0.3309 - accuracy: 0.8717 - val_loss: 0.3413 - val_accuracy: 0.8690
Epoch 5/50
125/125 [==============================] - 1s 11ms/step - loss: 0.2861 - accuracy: 0.8910 - val_loss: 0.3214 - val_accuracy: 0.8840
Epoch 6/50
125/125 [==============================] - 2s 13ms/step - loss: 0.2698 - accuracy: 0.8963 - val_loss: 0.2826 - val_accuracy: 0.8850
Epoch 7/50
125/125 [==============================] - 1s 12ms/step - loss: 0.2658 - accuracy: 0.8980 - val_loss: 0.2826 - val_accuracy: 0.8820
Epoch 8/50

In [8]:
# Save the trained model to an HDF5 file
model.save("model.h5")